In [1]:
import numpy as np
import sympy as sp 
from sympy import assoc_legendre
import matplotlib.pyplot as plt

In [2]:
l,m = sp.symbols('l,m',real=True)
x,y,z = sp.symbols('x,y,z',real=True,positive=True)
r,theta,phi = sp.symbols('r,theta,phi')

# function to convert spherical coords to cartesian 
def SphericalToCartesian(f):
  return f.subs([(r,sp.sqrt(x**2 + y**2 + z**2)),
                 (theta,sp.acos(z/sp.sqrt(x**2 + y**2 + z**2))),
                 (phi,sp.asin(y/sp.sqrt(x**2 + y**2)))])

In [3]:
# function to generate basis function in cartisian coords 
def basisFunction(l,m):
  l+=1
  c_plus = sp.factorial(l-1) * (-2)**(sp.Abs(m)) /sp.factorial(l + m) * sp.cos(m * phi)
  c_negt = sp.factorial(l-1) * (-2)**(sp.Abs(m)) /sp.factorial(l + sp.Abs(m)) * sp.sin(sp.Abs(m) * phi)

  s_plus  = SphericalToCartesian(c_plus* r**l * assoc_legendre(l,m,sp.cos(theta)))
  s_minus = SphericalToCartesian(c_negt* r**l * assoc_legendre(l,sp.Abs(m),sp.cos(theta)))
  
  if m >= 0:
    return [sp.expand_trig(sp.diff(s_plus, axis)).simplify() for axis in [x,y,z]]
  else:
    return [sp.expand_trig(sp.diff(s_minus, axis)).simplify().expand() for axis in [x,y,z]]

In [4]:
# function to evaluate a given basis function (l,m) at given cartisian coordinate point(u,v,z)
def get_basis_fv(u,v,w,l,m):
    bf = basisFunction(l,m)
    bfv = sp.lambdify([x,y,z],bf)
    return bfv(u,v,w)

In [5]:
# fucntion to filter coordinates from a grid using boolean masking  
def get_selected_coords(xv,yv,zv,bool_mask):
    x_coords = xv[bool_mask]
    y_coords = yv[bool_mask]
    z_coords = zv[bool_mask] 
    return np.array([x_coords,y_coords,z_coords])

In [6]:
# function to evaluate a given harmonics on selected coordinate mesh 
def get_harmonics_on_selected(xv,yv,zv,bool_mask,l,m):
    xx,yy,zz = get_selected_coords(xv,yv,zv,bool_mask)
    fvx,fvy,fvz  = get_basis_fv(xx,yy,zz,l,m)
    return [fvx,fvy,fvz]

In [7]:
# helper function to get the index of array where a fucntion is max 
def get_idx_maxf(func, pos_bool):
    max_val = np.max(func[pos_bool])
    idx = np.where(func[pos_bool]==max_val)[0]
    return idx

# this finds the x,y and z coordinates where the function is max on given surface 
def get_coords_maxf(func,pos_bool,xv,yv,zv):
    idx = get_idx_maxf(func,pos_bool)
    x_coords = xv[pos_bool][idx]
    y_coords = yv[pos_bool][idx]
    z_coords = zv[pos_bool][idx]
    return np.array([x_coords,y_coords,z_coords]).T
    

In [8]:
# basis function with l=2 and m=3 
basisFunction(2,3)

[x**2 - y**2, -2*x*y, 0]

In [9]:
# test above function 
get_basis_fv(2,1,0,2,3)

[3, -4, 0]

In [10]:
# create a mesh grid to evalute basis function 
length = np.linspace(-1,1,11)
xv,yv,zv= np.meshgrid(length,length,length)

In [11]:
# get the (x,y,z) components of basis function on a grid 
fvx,fvy,fvz  = get_basis_fv(xv,yv,zv,2,3)

In [12]:
# define boolean mask on the surface of cube 
surface_bool = (xv==1) + (xv==-1) + (yv==1) + (yv==-1) + (zv==1) + (zv ==-1)

In [13]:
max_coords = get_coords_maxf(fvy,surface_bool,xv,yv,zv)
max_coords

array([[ 1. , -1. , -1. ],
       [ 1. , -1. , -0.8],
       [ 1. , -1. , -0.6],
       [ 1. , -1. , -0.4],
       [ 1. , -1. , -0.2],
       [ 1. , -1. ,  0. ],
       [ 1. , -1. ,  0.2],
       [ 1. , -1. ,  0.4],
       [ 1. , -1. ,  0.6],
       [ 1. , -1. ,  0.8],
       [ 1. , -1. ,  1. ],
       [-1. ,  1. , -1. ],
       [-1. ,  1. , -0.8],
       [-1. ,  1. , -0.6],
       [-1. ,  1. , -0.4],
       [-1. ,  1. , -0.2],
       [-1. ,  1. ,  0. ],
       [-1. ,  1. ,  0.2],
       [-1. ,  1. ,  0.4],
       [-1. ,  1. ,  0.6],
       [-1. ,  1. ,  0.8],
       [-1. ,  1. ,  1. ]])

In [17]:
# top surface 
top_bool = zv==1
Bx,By,Bz = get_harmonics_on_selected(xv,yv,zv,top_bool,2,3)


In [18]:
Bx.shape

(121,)